# Projekt sieci neuronowej przewidującej ceny akcji - część techniczna

## Eksploaracyjna analiza danych

In [90]:
from typing import Any, Dict, Pattern, Set, Union, List
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorboard.plugins.hparams import api as hp
import yfinance as yf

In [104]:
# Załadowanie danych o indeksie S&P 500
gspc = yf.Ticker("^GSPC")
data_raw = gspc.history(period="max")

In [101]:
data_raw

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1963-01-18 00:00:00-05:00,0.000000,65.699997,64.860001,65.180000,4760000,0.0,0.0
1963-01-21 00:00:00-05:00,0.000000,65.519997,64.639999,65.279999,4090000,0.0,0.0
1963-01-22 00:00:00-05:00,0.000000,65.800003,65.029999,65.440002,4810000,0.0,0.0
1963-01-23 00:00:00-05:00,0.000000,65.910004,65.230003,65.620003,4820000,0.0,0.0
1963-01-24 00:00:00-05:00,0.000000,66.089996,65.330002,65.750000,4810000,0.0,0.0
...,...,...,...,...,...,...,...
2023-01-10 00:00:00-05:00,3888.570068,3919.830078,3877.290039,3919.250000,3851030000,0.0,0.0
2023-01-11 00:00:00-05:00,3932.350098,3970.070068,3928.540039,3969.610107,4303360000,0.0,0.0
2023-01-12 00:00:00-05:00,3977.570068,3997.760010,3937.560059,3983.169922,4440260000,0.0,0.0


In [105]:
# Wyświetlenie informacji o obieckie DataFrame
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23875 entries, 1927-12-30 00:00:00-05:00 to 2023-01-17 00:00:00-05:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          23875 non-null  float64
 1   High          23875 non-null  float64
 2   Low           23875 non-null  float64
 3   Close         23875 non-null  float64
 4   Volume        23875 non-null  int64  
 5   Dividends     23875 non-null  float64
 6   Stock Splits  23875 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


In [106]:
# Wyświetlenie statystyki opisowej
data_raw.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,23875.000000,23875.000000,23875.000000,23875.000000,2.387500e+04,23875.0,23875.0
mean,551.213345,574.765128,567.688081,571.447587,8.564998e+08,0.0,0.0
std,916.542347,910.457103,899.502505,905.312135,1.579968e+09,0.0,0.0
min,0.000000,4.400000,4.400000,4.400000,0.000000e+00,0.0,0.0
25%,9.480000,24.350000,24.350000,24.350000,1.420000e+06,0.0,0.0
50%,39.480000,101.919998,100.349998,101.089996,1.887000e+07,0.0,0.0
75%,942.294983,950.984985,932.700012,942.364990,7.632000e+08,0.0,0.0
max,4804.509766,4818.620117,4780.040039,4796.560059,1.145623e+10,0.0,0.0


In [107]:
data_raw = data_raw[["Open", "High", "Low", "Close", "Volume"]]

In [110]:
# Wyświetlenie ilośći brakujących wartośći -> nie ma takich wartości, nie rzeba uzupełniać
nan_count = data_raw.isna().sum()
print(nan_count)

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [123]:
# Ilość wierszy, w których wartości są większe od zera
data_raw[data_raw > 0].count()

Open      18800
High      23875
Low       23875
Close     23875
Volume    18379
dtype: int64

In [125]:
# Jak widać w kolumnach Open i Volume znajdują się wiersze z wartościami 0, czyli są niekompletne
# W takim przypadku bierzemy zakres danych po 1982-04-20, gdzie wszystkie wiersze mają wartości większe od zera
data_raw_full = data_raw["1982-04-20 00:00:00-05:00":]
data_raw_full[data_raw_full > 0].count()

Open      10273
High      10273
Low       10273
Close     10273
Volume    10273
dtype: int64

In [126]:
fig = px.line(data_raw_full, x=data_raw_full.index, y=["Open", "Close"], line_shape="spline", labels={"variable":"Legenda", "value":"Cena"}, 
            render_mode="svg", title="Wykres wartości otwarcia i zamknięcia indeksu S&P500 w czasie")
newnames = {"Open":"Cena otwarcia", "Close":"Cena zamknięcia"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

In [128]:
fig = px.area(data_raw_full, x=data_raw_full.index, y="Volume", line_shape="spline", 
             title="Wykres ilości transakcji indeksu S&P500 w czasie")
fig.show()

## Projektowanie sieci neuronowej

### Trening podstawowej sieci neuronowej

In [130]:
def generate_features(df):
    """
    Funkcja generująca cechy na podstawie historycznych wartości indeksu i jego zmienności
    @param df: obiekt DataFrame zawierający kolumny "Open", "Close", "High", "Low", "Volume"
    @return: obiekt DataFrame zawierający zbiór danych z nowymi cechami    
    """
    df_new = pd.DataFrame()
    # 6 oryginalnych cech
    df_new['open'] = df['Open']
    df_new['open_1'] = df['Open'].shift(1)
    df_new['close_1'] = df['Close'].shift(1)
    df_new['high_1'] = df['High'].shift(1)
    df_new['low_1'] = df['Low'].shift(1)
    df_new['volume_1'] = df['Volume'].shift(1)
    # 31 wygenerowanych cech
    # Średnie ceny
    df_new['avg_price_5'] = df['Close'].rolling(5).mean().shift(1)
    df_new['avg_price_30'] = df['Close'].rolling(21).mean().shift(1)
    df_new['avg_price_365'] = df['Close'].rolling(252).mean().shift(1)
    df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
    df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
    df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']
    # Średnie woluminy
    df_new['avg_volume_5'] = df['Volume'].rolling(5).mean().shift(1)
    df_new['avg_volume_30'] = df['Volume'].rolling(21).mean().shift(1)
    df_new['avg_volume_365'] = df['Volume'].rolling(252).mean().shift(1)
    df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
    df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
    df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']
    # Odchylenia standardowe cen
    df_new['std_price_5'] = df['Close'].rolling(5).std().shift(1)
    df_new['std_price_30'] = df['Close'].rolling(21).std().shift(1)
    df_new['std_price_365'] = df['Close'].rolling(252).std().shift(1)
    df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
    df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
    df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']
    # Odchylenia standardowe woluminów
    df_new['std_volume_5'] = df['Volume'].rolling(5).std().shift(1)
    df_new['std_volume_30'] = df['Volume'].rolling(21).std().shift(1)
    df_new['std_volume_365'] = df['Volume'].rolling(252).std().shift(1)
    df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
    df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
    df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']
    # Zwroty
    df_new['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
    df_new['return_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
    df_new['return_30'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
    df_new['return_365'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)
    df_new['moving_avg_5'] = df_new['return_1'].rolling(5).mean().shift(1)
    df_new['moving_avg_30'] = df_new['return_1'].rolling(21).mean().shift(1)
    df_new['moving_avg_365'] = df_new['return_1'].rolling(252).mean().shift(1)
    # Wartości docelowe
    df_new['close'] = df['Close']
    df_new = df_new.dropna(axis=0)
    return df_new

In [131]:
# Wywołanie funkcji generate_features, która generuje cechy i etykiety
data = generate_features(data_raw_full)

In [132]:
# Ustalenie zakresów dat, na które ma być podzielony zbiór danych
start_train = '20-04-1982'
end_train = '31-12-2021'
start_test = '03-01-2022'
end_test = '17-01-2023'

# Utworzenie zbioru treningowego i zbioru testowego
data_train = data.loc[start_train:end_train]
X_train = data_train.drop('close', axis=1).values
y_train = data_train['close'].values
data_test = data.loc[start_test:end_test]
X_test = data_test.drop('close', axis=1).values
y_test = data_test['close'].values

/tmp/ipykernel_9253/3301436121.py:8: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.

/tmp/ipykernel_9253/3301436121.py:11: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.



In [133]:
# Znormalizowanie cech, aby miały tę samą lub porównywalną skalę. 
# Polega to na odjęciu od nich średniej wartości i przeskalowaniu ich do jednostkowej wariancji.
# Do przeskalowania zbiorów użyto przetrenowanego obiektu scaler
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [134]:
# Ustalenie seedu w celu testowania
tf.random.set_seed(42)

In [211]:
# Zbudowanie sieci neuronowej, wykorzystującą klasę Sequential zawartą w bibliotece Keras. 
# Początkowa sieć składa się z jednej warstwy ukrytej, 
# zbudowanej z 32 węzłów i wykorzystującej funkcję aktywacji ReLU
default_model = Sequential([
    Dense(units=32, activation='relu'),
    Dense(units=1)
])

# Skomplilowanie modelu sieci, wykorzystując optymalizator Adam. 
# Przyjęta szybkość uczenia się 0.1 i błąd średniokwadratowy jako cel treningu.
default_model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.1))
default_model.fit(X_scaled_train, y_train, epochs=100, verbose=True)
predictions = default_model.predict(X_scaled_test)[:, 0]

# Użycie przetrenowanego modelu do przetworzenia zbioru testowego i wyświetlenie wskaźników skuteczności
print(f'Błąd średniokwadratowy: {mean_squared_error(y_test, predictions):.3f}')
print(f'Średni błąd bezwzględny: {mean_absolute_error(y_test, predictions):.3f}')
print(f'R^2: {r2_score(y_test, predictions):.3f}')

Epoch 1/100
305/305 [==============================] - 5s 3ms/step - loss: 193551.7656
Epoch 2/100
305/305 [==============================] - 1s 3ms/step - loss: 2665.5161
Epoch 3/100
305/305 [==============================] - 1s 2ms/step - loss: 997.3510
Epoch 4/100
305/305 [==============================] - 1s 2ms/step - loss: 1052.2070
Epoch 5/100
305/305 [==============================] - 1s 3ms/step - loss: 845.0105
Epoch 6/100
305/305 [==============================] - 1s 3ms/step - loss: 956.1525
Epoch 7/100
305/305 [==============================] - 1s 3ms/step - loss: 790.3945
Epoch 8/100
305/305 [==============================] - 1s 2ms/step - loss: 1006.0798
Epoch 9/100
305/305 [==============================] - 1s 2ms/step - loss: 1088.6803
Epoch 10/100
305/305 [==============================] - 1s 2ms/step - loss: 899.7809
Epoch 11/100
305/305 [==============================] - 1s 3ms/step - loss: 943.1082
Epoch 12/100
305/305 [==============================] - 1s 3ms/step

In [212]:
# Zapis modelu
default_model.save("models/default_model")

INFO:tensorflow:Assets written to: models/default_model/assets


INFO:tensorflow:Assets written to: models/default_model/assets


### Dostrojenie parametrów sieci neuronowej za pomocą modułu hparams

In [136]:
# Dostosowywana będzie liczba węzłów w ukrytej warstwie, liczbę warstw ukrytych plus wyjściowa, liczba iteracji treningowych i szybkość uczenia.
# Będą to trzy liczby węzłów(wartości dyskretne) równe 13, 32 i 64,
# trzy liczby warstw ukrytych plus wyjściowa równe 2, 3, 5 
# dwie liczby iteracji(wartości dyskretne) równe 300 i 1000 oraz 
# szybkość uczenia(wartość ciągła) z zakresu od 0,01 do 4

HP_HIDDEN = hp.HParam('hidden_size', hp.Discrete([64, 32, 16]))
HP_LAYERS = hp.HParam('layers', hp.Discrete([2, 3, 5]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([300, 1000]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.01, 0.4))

In [137]:
def train_test_model(hparams: Dict[hp.HParam, Any], logdir: str) -> tuple:
    """
    Funkcja trenująca i testująca model sieci neuronowej
    @param hparams: obiekt słownika zawierający hiperparametry w postaci {nazwa_hiperparametru:wartość}
    @param logdir: ścieżka dla każdej iteracji trenowania i testowania sieci
    @return: mse, r2 - obiekt tuple zawierający wskażniki skuteczności modelu    
    """
    # Założono trenowanie i testowanie sieci neuronowej o 3 architekturach:
    #   - jedna warstwa ukryta
    #   - dwie warstwy ukryte, druga z połową węzłów
    #   - dwie warstwy ukryte, druga z połową węzłów, dodatkowo zostosowamy Dropout o wartości 0.5
    if hparams[HP_LAYERS] == 2:
        layers_list = [Dense(units=hparams[HP_HIDDEN], activation='relu'), Dense(units=1)]
    elif hparams[HP_LAYERS] == 3:
        layers_list = [Dense(units=hparams[HP_HIDDEN], activation='relu'), Dense(units=(hparams[HP_HIDDEN]//2), activation='relu'), Dense(units=1)]
    elif hparams[HP_LAYERS] == 5:
        layers_list = [Dense(units=hparams[HP_HIDDEN], activation='relu'), Dropout(0.5), Dense(units=(hparams[HP_HIDDEN]//2), activation='relu'), Dropout(0.5), Dense(units=1)]

    model = Sequential(layers_list)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(hparams[HP_LEARNING_RATE]),
                  metrics=['mean_squared_error'])
    model.fit(X_scaled_train, y_train, validation_data=(X_scaled_test, y_test), epochs=hparams[HP_EPOCHS], verbose=False,
              callbacks=[
                  tf.keras.callbacks.TensorBoard(logdir),
                  hp.KerasCallback(logdir, hparams),
                  tf.keras.callbacks.EarlyStopping(
                      monitor='val_loss', min_delta=0, patience=200, verbose=0, mode='auto',
                  )
              ],
              )
    _, mse = model.evaluate(X_scaled_test, y_test)
    pred = model.predict(X_scaled_test)
    r2 = r2_score(y_test, pred)
    return mse, r2

In [138]:
def run(hparams: Dict[hp.HParam, Any], logdir: str) -> None:
    """
    Funkcja inicjująca proces treningu za pomocą kombinacji hiperparametrów oraz wyświetlająca podsumowanie 
    zawierające wartości błędu średniokwadratowego i współczynnika R^2
    @param hparams: obiekt słownika zawierający hiperparametry w postaci {nazwa_hiperparametru:wartość}
    @param logdir: ścieżka dla każdej iteracji trenowania i testowania sieci   
    """
    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(
            hparams=[HP_HIDDEN, HP_LAYERS, HP_EPOCHS, HP_LEARNING_RATE],
            metrics=[hp.Metric('mean_squared_error', display_name='mse'),
                     hp.Metric('r2', display_name='r2'),
                    ],
        )
        mse, r2 = train_test_model(hparams, logdir)
        tf.summary.scalar('mean_squared_error', mse, step=1)
        tf.summary.scalar('r2', r2, step=1)

In [140]:
# Tworzenie, kompilowanie i trenowanie modelu sieci neuronowej z wszystkimi możliwymi kombinacjami hiperparametrów.
# W każdej próbie są stosowane trzy wartości dyskretne(liczbę węzłów w warstwie ukrytej, liczbę warstw i liczbę iteracji) wybrane ze zdefiniowanych pól 
# oraz jedna wartość ciągła(szybkość uczenia) wybrana z jednego z równych podprzedziałów z założonego zakresu
session_num = 0
for hidden in HP_HIDDEN.domain.values:
    for layers_number in HP_LAYERS.domain.values:
        for epochs in HP_EPOCHS.domain.values:
            for learning_rate in tf.linspace(HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value, 5):
                hparams = {
                    HP_HIDDEN: hidden,
                    HP_LAYERS: layers_number,
                    HP_EPOCHS: epochs,
                    HP_LEARNING_RATE: float("%.2f"%float(learning_rate)),
                }
                # W chwili rozpoczęcia prób tworzone są katalogi, 
                # w których zapisywane są wyniki treningu i weryfikacji modelu w każdej próbie.
                run_name = "run-%d" % session_num
                print('--- Próba: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run(hparams, 'logs/hparam_tuning/' + run_name)
                session_num += 1



--- Próba: run-0
{'hidden_size': 16, 'layers': 2, 'epochs': 300, 'learning_rate': 0.01}
7/7 [==============================] - 0s 734us/step
--- Próba: run-1
{'hidden_size': 16, 'layers': 2, 'epochs': 300, 'learning_rate': 0.11}
7/7 [==============================] - 0s 897us/step
--- Próba: run-2
{'hidden_size': 16, 'layers': 2, 'epochs': 300, 'learning_rate': 0.21}
7/7 [==============================] - 0s 773us/step
--- Próba: run-3
{'hidden_size': 16, 'layers': 2, 'epochs': 300, 'learning_rate': 0.3}
7/7 [==============================] - 0s 1ms/step
--- Próba: run-4
{'hidden_size': 16, 'layers': 2, 'epochs': 300, 'learning_rate': 0.4}
7/7 [==============================] - 0s 2ms/step
--- Próba: run-5
{'hidden_size': 16, 'layers': 2, 'epochs': 1000, 'learning_rate': 0.01}
7/7 [==============================] - 0s 708us/step
--- Próba: run-6
{'hidden_size': 16, 'layers': 2, 'epochs': 1000, 'learning_rate': 0.11}
7/7 [==============================] - 0s 841us/step
--- Próba: run-7


Po przeprowadzeniu wszystkich prób można otworzyć panel Tensorboard za pomocą polecenia:

`tensorboard --logdir logs/hparam_tuning`

W zakładce HPARAMS możemy zobaczyć tabele zawierającą wszystkie kombinacje hiperparametrów i opowiadające im wskaźniki skuteczności:

![TensorBoard](./data/TensorBoard_hparams_top_r2.png)

Jak widać najlepszą skuteczność model osiąga dla kombinacji hiperparametrów layers=3, hidden_size=64, epochs=1000 i learning_rate=0.4, dla której współczynnik R^2 równy jest 0.955


In [213]:
# Użycie optymalnego modelu do wyliczenia prognoz
final_model = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=1)
])

final_model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.4))
final_model.fit(X_scaled_train, y_train, epochs=1000, verbose=False)
predictions = final_model.predict(X_scaled_test)[:, 0]
print(f'Błąd średniokwadratowy: {mean_squared_error(y_test, predictions):.3f}')
print(f'Średni błąd bezwzględny: {mean_absolute_error(y_test, predictions):.3f}')
print(f'R^2: {r2_score(y_test, predictions):.3f}')

7/7 [==============================] - 0s 4ms/step
Błąd średniokwadratowy: 2769.812
Średni błąd bezwzględny: 42.041
R^2: 0.952


In [143]:
# Utworzenie wykresu wartości prognozowanych i rzeczywistych
fig = px.line(x=data_test.index, y=[y_test, predictions], line_shape="spline", labels={"variable":"Legenda", "value":"Cena zamknięcia", "x":"Data"}, 
            render_mode="svg", title="Wykres wartości rzeczywistych i prognozowanych indeksu S&P500 w czasie")
newnames = {"wide_variable_0":"Wartości rzeczywiste", "wide_variable_1":"Prognozy sieci neuronowej"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

In [214]:
# Zapis modelu
final_model.save("models/final_model")


INFO:tensorflow:Assets written to: models/final_model/assets


INFO:tensorflow:Assets written to: models/final_model/assets
